In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from core.yolov4 import YOLOv4, decode_train

In [8]:
def train_model(input_size,num_class, just_summary=True):
    
    input_layer = layers.Input(shape=(input_size, input_size, 3))
    
    feature_maps = YOLOv4(input_layer, num_class)
    ANCHORS = [12,16, 19,36, 40,28, 36,75, 76,55, 72,146, 142,110, 192,243, 459,401]
    STRIDES = [8, 16, 32]
    XYSCALE = [1.2, 1.1, 1.05]
    bbox_tensors = []
    for i, fm in enumerate(feature_maps):
        if i == 0:
            bbox_tensor = decode_train(fm, input_size // 8, num_class, STRIDES, ANCHORS, i, XYSCALE)
        elif i == 1:
            bbox_tensor = decode_train(fm, input_size // 16, num_class, STRIDES, ANCHORS, i, XYSCALE)
        else:
            bbox_tensor = decode_train(fm, input_size // 32, num_class, STRIDES, ANCHORS, i, XYSCALE)
        bbox_tensors.append(fm)
        bbox_tensors.append(bbox_tensor)
            
    model = tf.keras.Model(input_layer, bbox_tensors)
    if just_summary:
        print(model.summary())
        return model
    optimizer = tf.keras.optimizers.Adam()
    if os.path.exists(logdir): shutil.rmtree(logdir)
    writer = tf.summary.create_file_writer(logdir)

    # define training step function
    # @tf.function
    def train_step(image_data, target):
        with tf.GradientTape() as tape:
            pred_result = model(image_data, training=True)
            giou_loss = conf_loss = prob_loss = 0

            # optimizing process
            for i in range(len(freeze_layers)):
                conv, pred = pred_result[i * 2], pred_result[i * 2 + 1]
                loss_items = compute_loss(pred, conv, target[i][0], target[i][1], STRIDES=STRIDES, NUM_CLASS=NUM_CLASS, IOU_LOSS_THRESH=IOU_LOSS_THRESH, i=i)
                giou_loss += loss_items[0]
                conf_loss += loss_items[1]
                prob_loss += loss_items[2]

            total_loss = giou_loss + conf_loss + prob_loss

            gradients = tape.gradient(total_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            tf.print("=> STEP %4d/%4d   lr: %.6f   giou_loss: %4.2f   conf_loss: %4.2f   "
                     "prob_loss: %4.2f   total_loss: %4.2f" % (global_steps, total_steps, optimizer.lr.numpy(),
                                                               giou_loss, conf_loss,
                                                               prob_loss, total_loss))
            # update learning rate
            global_steps.assign_add(1)
            if global_steps < warmup_steps:
                lr = global_steps / warmup_steps * cfg.TRAIN.LR_INIT
            else:
                lr = cfg.TRAIN.LR_END + 0.5 * (cfg.TRAIN.LR_INIT - cfg.TRAIN.LR_END) * (
                    (1 + tf.cos((global_steps - warmup_steps) / (total_steps - warmup_steps) * np.pi))
                )
            optimizer.lr.assign(lr.numpy())

            # writing summary data
            with writer.as_default():
                tf.summary.scalar("lr", optimizer.lr, step=global_steps)
                tf.summary.scalar("loss/total_loss", total_loss, step=global_steps)
                tf.summary.scalar("loss/giou_loss", giou_loss, step=global_steps)
                tf.summary.scalar("loss/conf_loss", conf_loss, step=global_steps)
                tf.summary.scalar("loss/prob_loss", prob_loss, step=global_steps)
            writer.flush()
    def test_step(image_data, target):
        with tf.GradientTape() as tape:
            pred_result = model(image_data, training=True)
            giou_loss = conf_loss = prob_loss = 0

            # optimizing process
            for i in range(len(freeze_layers)):
                conv, pred = pred_result[i * 2], pred_result[i * 2 + 1]
                loss_items = compute_loss(pred, conv, target[i][0], target[i][1], STRIDES=STRIDES, NUM_CLASS=NUM_CLASS, IOU_LOSS_THRESH=IOU_LOSS_THRESH, i=i)
                giou_loss += loss_items[0]
                conf_loss += loss_items[1]
                prob_loss += loss_items[2]

            total_loss = giou_loss + conf_loss + prob_loss

            tf.print("=> TEST STEP %4d   giou_loss: %4.2f   conf_loss: %4.2f   "
                     "prob_loss: %4.2f   total_loss: %4.2f" % (global_steps, giou_loss, conf_loss,
                                                               prob_loss, total_loss))

    # ----------------Training---- 
    for epoch in range(first_stage_epochs + second_stage_epochs):
        if epoch < first_stage_epochs:
            if not isfreeze:
                isfreeze = True
                for name in freeze_layers:
                    freeze = model.get_layer(name)
                    freeze_all(freeze)
        elif epoch >= first_stage_epochs:
            if isfreeze:
                isfreeze = False
                for name in freeze_layers:
                    freeze = model.get_layer(name)
                    unfreeze_all(freeze)
        for image_data, target in trainset:
            train_step(image_data, target)
        for image_data, target in testset:
            test_step(image_data, target)
        model.save_weights("./checkpoints/yolov4")
        
        return 2
    
    

In [9]:
model = train_model(300,5)

------------------------------
conv sahpe KerasTensor(type_spec=TensorSpec(shape=(None, 18, 18, 512), dtype=tf.float32, name=None), name='tf.nn.leaky_relu_36/LeakyRelu:0', description="created by layer 'tf.nn.leaky_relu_36'")
route_1 shape: KerasTensor(type_spec=TensorSpec(shape=(None, 37, 37, 256), dtype=tf.float32, name=None), name='tf.math.multiply_80/Mul:0', description="created by layer 'tf.math.multiply_80'")
conv sahpe (None, 18, 18, 512)
route_1 shape: (None, 37, 37, 256)
conv sahpe (None, 18, 18, 256)
conv sahpe (None, 18, 18, 256)
route_1 shape: (None, 36, 36, 128)
Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
     

                                                                                                  
 tf.math.tanh_51 (TFOpLambda)   (None, 150, 150, 64  0           ['tf.math.softplus_51[0][0]']    
                                )                                                                 
                                                                                                  
 tf.math.multiply_60 (TFOpLambd  (None, 150, 150, 64  0          ['batch_normalization_85[0][0]', 
 a)                             )                                 'tf.math.tanh_51[0][0]']        
                                                                                                  
 tf.__operators__.add_14 (TFOpL  (None, 150, 150, 64  0          ['tf.math.multiply_58[0][0]',    
 ambda)                         )                                 'tf.math.multiply_60[0][0]']    
                                                                                                  
 conv2d_88

                                                                                                  
 tf.math.multiply_65 (TFOpLambd  (None, 75, 75, 64)  0           ['batch_normalization_90[0][0]', 
 a)                                                               'tf.math.tanh_56[0][0]']        
                                                                                                  
 conv2d_93 (Conv2D)             (None, 75, 75, 64)   4096        ['tf.math.multiply_65[0][0]']    
                                                                                                  
 batch_normalization_91 (BatchN  (None, 75, 75, 64)  256         ['conv2d_93[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.math.softplus_57 (TFOpLambd  (None, 75, 75, 64)  0           ['batch_normalization_91[0][0]'] 
 a)       

 a)                                                               'tf.math.tanh_61[0][0]']        
                                                                                                  
 tf.math.multiply_64 (TFOpLambd  (None, 75, 75, 64)  0           ['batch_normalization_89[0][0]', 
 a)                                                               'tf.math.tanh_55[0][0]']        
                                                                                                  
 tf.concat_17 (TFOpLambda)      (None, 75, 75, 128)  0           ['tf.math.multiply_70[0][0]',    
                                                                  'tf.math.multiply_64[0][0]']    
                                                                                                  
 conv2d_98 (Conv2D)             (None, 75, 75, 128)  16384       ['tf.concat_17[0][0]']           
                                                                                                  
 batch_nor

                                                                                                  
 tf.math.softplus_68 (TFOpLambd  (None, 37, 37, 128)  0          ['batch_normalization_102[0][0]']
 a)                                                                                               
                                                                                                  
 tf.math.tanh_68 (TFOpLambda)   (None, 37, 37, 128)  0           ['tf.math.softplus_68[0][0]']    
                                                                                                  
 tf.math.multiply_77 (TFOpLambd  (None, 37, 37, 128)  0          ['batch_normalization_102[0][0]',
 a)                                                               'tf.math.tanh_68[0][0]']        
                                                                                                  
 conv2d_105 (Conv2D)            (None, 37, 37, 128)  147456      ['tf.math.multiply_77[0][0]']    
          

...

                                                                                                  
 batch_normalization_141 (Batch  (None, 36, 36, 128)  512        ['conv2d_143[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 tf.nn.leaky_relu_48 (TFOpLambd  (None, 36, 36, 128)  0          ['batch_normalization_141[0][0]']
 a)                                                                                               
                                                                                                  
 conv2d_144 (Conv2D)            (None, 36, 36, 256)  294912      ['tf.nn.leaky_relu_48[0][0]']    
                                                                                                  
 batch_normalization_142 (Batch  (None, 36, 36, 256)  1024       ['conv2d_144[0][0]']             
 Normaliza

 Normalization)                                                                                   
                                                                                                  
 tf.nn.leaky_relu_59 (TFOpLambd  (None, 9, 9, 512)   0           ['batch_normalization_152[0][0]']
 a)                                                                                               
                                                                                                  
 tf.image.resize_3 (TFOpLambda)  (None, 18, 18, 512)  0          ['tf.nn.leaky_relu_59[0][0]']    
                                                                                                  
 tf.concat_25 (TFOpLambda)      (None, 18, 18, 1024  0           ['tf.image.resize_3[0][0]',      
                                )                                 'tf.nn.leaky_relu_36[0][0]']    
                                                                                                  
 conv2d_15

 tf.compat.v1.shape_6 (TFOpLamb  (4,)                0           ['conv2d_147[0][0]']             
 da)                                                                                              
                                                                                                  
 tf.compat.v1.shape_8 (TFOpLamb  (4,)                0           ['conv2d_155[0][0]']             
 da)                                                                                              
                                                                                                  
 tf.compat.v1.shape_10 (TFOpLam  (4,)                0           ['conv2d_163[0][0]']             
 bda)                                                                                             
                                                                                                  
 tf.__operators__.getitem_6 (Sl  ()                  0           ['tf.compat.v1.shape_6[0][0]']   
 icingOpLa

 da)                            )                                                                 
                                                                                                  
 tf.tile_3 (TFOpLambda)         (None, 37, 37, 3, 2  0           ['tf.__operators__.getitem_7[0][0
                                )                                ]']                              
                                                                                                  
 tf.math.multiply_104 (TFOpLamb  (None, 18, 18, 3, 2  0          ['tf.math.sigmoid_12[0][0]']     
 da)                            )                                                                 
                                                                                                  
 tf.tile_4 (TFOpLambda)         (None, 18, 18, 3, 2  0           ['tf.__operators__.getitem_9[0][0
                                )                                ]']                              
          

                                                                                                  
 tf.math.sigmoid_14 (TFOpLambda  (None, 18, 18, 3, 5  0          ['tf.split_4[0][3]']             
 )                              )                                                                 
                                                                                                  
 tf.concat_30 (TFOpLambda)      (None, 9, 9, 3, 4)   0           ['tf.math.multiply_108[0][0]',   
                                                                  'tf.math.multiply_109[0][0]']   
                                                                                                  
 tf.math.sigmoid_16 (TFOpLambda  (None, 9, 9, 3, 1)  0           ['tf.split_5[0][2]']             
 )                                                                                                
                                                                                                  
 tf.math.s

In [4]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 300, 300, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 300, 300, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 300, 300, 32  128        ['conv2d[0][0]']                 
 rmalization)                   )                                                             

                                                                                                  
 tf.__operators__.add (TFOpLamb  (None, 150, 150, 64  0          ['tf.math.multiply_3[0][0]',     
 da)                            )                                 'tf.math.multiply_5[0][0]']     
                                                                                                  
 conv2d_6 (Conv2D)              (None, 150, 150, 64  4096        ['tf.__operators__.add[0][0]']   
                                )                                                                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 150, 150, 64  4096        ['tf.math.multiply_1[0][0]']     
                                )                                                                 
                                                                                                  
 batch_nor

 batch_normalization_12 (BatchN  (None, 75, 75, 64)  256         ['conv2d_11[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.math.softplus_11 (TFOpLambd  (None, 75, 75, 64)  0           ['batch_normalization_12[0][0]'] 
 a)                                                                                               
                                                                                                  
 tf.math.tanh_11 (TFOpLambda)   (None, 75, 75, 64)   0           ['tf.math.softplus_11[0][0]']    
                                                                                                  
 tf.math.multiply_11 (TFOpLambd  (None, 75, 75, 64)  0           ['batch_normalization_12[0][0]', 
 a)                                                               'tf.math.tanh_11[0][0]']        
          

                                                                  'tf.math.multiply_9[0][0]']     
                                                                                                  
 conv2d_16 (Conv2D)             (None, 75, 75, 128)  16384       ['tf.concat_1[0][0]']            
                                                                                                  
 batch_normalization_17 (BatchN  (None, 75, 75, 128)  512        ['conv2d_16[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.math.softplus_16 (TFOpLambd  (None, 75, 75, 128)  0          ['batch_normalization_17[0][0]'] 
 a)                                                                                               
                                                                                                  
 tf.math.t

 tf.math.multiply_22 (TFOpLambd  (None, 37, 37, 128)  0          ['batch_normalization_23[0][0]', 
 a)                                                               'tf.math.tanh_22[0][0]']        
                                                                                                  
 conv2d_23 (Conv2D)             (None, 37, 37, 128)  147456      ['tf.math.multiply_22[0][0]']    
                                                                                                  
 batch_normalization_24 (BatchN  (None, 37, 37, 128)  512        ['conv2d_23[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.math.softplus_23 (TFOpLambd  (None, 37, 37, 128)  0          ['batch_normalization_24[0][0]'] 
 a)                                                                                               
          

...

                                                                                                  
 batch_normalization_63 (BatchN  (None, 36, 36, 256)  1024       ['conv2d_62[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_16 (TFOpLambd  (None, 36, 36, 256)  0          ['batch_normalization_63[0][0]'] 
 a)                                                                                               
                                                                                                  
 conv2d_63 (Conv2D)             (None, 36, 36, 128)  32768       ['tf.nn.leaky_relu_16[0][0]']    
                                                                                                  
 batch_normalization_64 (BatchN  (None, 36, 36, 128)  512        ['conv2d_63[0][0]']              
 ormalizat

                                )                                 'tf.nn.leaky_relu_3[0][0]']     
                                                                                                  
 conv2d_75 (Conv2D)             (None, 18, 18, 512)  524288      ['tf.concat_9[0][0]']            
                                                                                                  
 batch_normalization_74 (BatchN  (None, 18, 18, 512)  2048       ['conv2d_75[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 tf.nn.leaky_relu_27 (TFOpLambd  (None, 18, 18, 512)  0          ['batch_normalization_74[0][0]'] 
 a)                                                                                               
                                                                                                  
 conv2d_76

                                                                                                  
 tf.__operators__.getitem (Slic  ()                  0           ['tf.compat.v1.shape[0][0]']     
 ingOpLambda)                                                                                     
                                                                                                  
 tf.__operators__.getitem_2 (Sl  ()                  0           ['tf.compat.v1.shape_2[0][0]']   
 icingOpLambda)                                                                                   
                                                                                                  
 tf.__operators__.getitem_4 (Sl  ()                  0           ['tf.compat.v1.shape_4[0][0]']   
 icingOpLambda)                                                                                   
                                                                                                  
 tf.reshap

                                )                                ]']                              
                                                                                                  
 tf.math.multiply_52 (TFOpLambd  (None, 9, 9, 3, 2)  0           ['tf.math.sigmoid_6[0][0]']      
 a)                                                                                               
                                                                                                  
 tf.tile_2 (TFOpLambda)         (None, 9, 9, 3, 2)   0           ['tf.__operators__.getitem_5[0][0
                                                                 ]']                              
                                                                                                  
 tf.math.subtract (TFOpLambda)  (None, 37, 37, 3, 2  0           ['tf.math.multiply_46[0][0]']    
                                )                                                                 
          

 tf.math.sigmoid_8 (TFOpLambda)  (None, 9, 9, 3, 5)  0           ['tf.split_2[0][3]']             
                                                                                                  
 tf.concat_11 (TFOpLambda)      (None, 37, 37, 3, 1  0           ['tf.concat_10[0][0]',           
                                0)                                'tf.math.sigmoid_1[0][0]',      
                                                                  'tf.math.sigmoid_2[0][0]']      
                                                                                                  
 tf.concat_13 (TFOpLambda)      (None, 18, 18, 3, 1  0           ['tf.concat_12[0][0]',           
                                0)                                'tf.math.sigmoid_4[0][0]',      
                                                                  'tf.math.sigmoid_5[0][0]']      
                                                                                                  
 tf.concat